In [17]:
import pandas as pd
import numpy as np
from gensim.models import Word2Vec

path = "./물금고등학교-익명의-숲_진로편/"


In [12]:
df_morphs = pd.read_csv(path + 'morphs.csv', converters={'morph': eval})
morphs = df_morphs['morph'].to_list()

df_cat = pd.read_csv(path + 'data/category.csv')

In [13]:
word2vec = Word2Vec(morphs, min_count=5)

In [16]:
type_sim_words = dict()
for type_name, type_words in df_cat.iteritems():
    type_sim_words[type_name] = []
    for type_word in type_words:
        type_sim_words[type_name].extend([type_word])
        if type_word in word2vec.wv:
            type_sim_words[type_name].extend(np.array(word2vec.wv.most_similar(type_word, topn=5))[:, 0])
            
            # print(type_word, type_sim_words[-1])
    print(len(type_sim_words[type_name]), len(set(type_sim_words[type_name])))
    type_sim_words[type_name] = list(set(type_sim_words[type_name]))[:80]

178 96
178 87
173 96
173 81
193 101
208 109


In [15]:
df_type = pd.DataFrame(type_sim_words, columns=df_cat.columns)
df_type.to_csv(path + 'data/category_sim.csv', index=False)


In [67]:
list_count = []
for idx, (writer, words) in df_morphs[['writer', 'morph']].iterrows():
    list_count.append([])
    for type_name, type_words in df_type.iteritems():
        count = 0
        for word in words:
            if word in word2vec.wv:
                sim_word = np.array(word2vec.wv.most_similar(word, topn=5))[:, 0]
                count += len(set(type_words) & set(sim_word))
                # if type_word in np.array(word2vec.wv.most_similar(word, topn=10))[:, 0]:
                #     print(word, type_word)
        list_count[-1].append(count)
            # print('점' in np.array(word2vec.wv.most_similar(word))[:, 0])
    # print(writer, list_count[-1])

In [68]:
df_count = df_morphs
df_count['count'] = list_count
df_count.drop(df_count[df_count['count'].apply(sum) < 100].index, inplace=True)


In [69]:
df_count.to_csv(path + 'count.csv', index=False)